In [ ]:
import random
import pandas as pd
from collections import defaultdict
import heapq
from collections import deque
import numpy as np

In [3]:
def generate_events(df):
    events = []

    for _, row in df.iterrows():
        ts = int(row.name.value // 1e6)
        mid = float((row.High + row.Low) / 2)

        #generate synthetic L2 activities:
        for _ in range(random.randint(3,8)):
            side = random.choice(['BUY', 'SELL'])
            price = round(mid + random.choice([-1, 1]) * random.uniform(0.1,0.5),2)
            qty = random.randint(10,100)
            etype = random.choice(['NEW','CANCEL','TRADE'])

            events.append((ts,side,price,qty,etype))
    return sorted(events, key = lambda x: x[0])



In [4]:
class OrderBook():
    def __init__(self):
        self.market_bids = defaultdict(float)
        self.market_asks = defaultdict(float)
        self.my_orders = {}

    def process(self,event):
        ts,side,price,qty,etype = event
        book = self.market_bids if side == 'BUY' else self.market_asks #it means the market book not mine buy means bids

        if etype == 'NEW':
            book[price] += qty
        elif etype == 'CANCEL':
            book[price] -= qty
            if book[price] <= 0:
                del book[price]
                
    def cancel_order(self,order_id):
        if order_id not in self.my_orders:
            return

        order = self.my_orders[order_id]
        side = order['side']
        price = order['price']
        qty = order['qty']
        book = self.bids if side == 'BUY' else self.asks

        #remove remaining orders:
        book[price] -= qty
        if book[price] <= 0:
            del book[price]

        del self.my_orders[order_id]
               
    def best_bid(self):
        return max(self.market_bids.keys(), default=None)
    def best_ask(self):
        return min(self.market_asks.keys(),default=None)

    def imbalance(self):
        bid_vol = sum(self.market_bids.values())
        ask_vol = sum(self.market_asks.values())
        if (bid_vol + ask_vol) == 0:
            return 0
        else:
            return (bid_vol - ask_vol) / (bid_vol + ask_vol)

        
    def place_limit_order(self,order_id,side,price,qty):
        book = self.market_bids if side == 'BUY' else self.market_asks #You are joining the bid at 100.00.  if side is BUY

        queue_ahead = book.get(price,0) #book = {100.00: 5, 99.99: 3}, it will return 5, There are already 5 contracts ahead of you at this price.

        self.my_orders[order_id] = {
            'side': side,
            'price': price,
            'qty': qty,
            'remaining_qty': qty,
            'queue_ahead': queue_ahead
        } #Now your system records your order: like price 100, qty 2, queue ahead 5, id 42 etc

        
    def process_trade(self,side,price,traded_qty): #suppose side=SELL, price=100 and qty 6, someone sold 6 contracts into the bid at 100
        filled_orders = [] #will stores fill here

        for oid, order in list(self.my_orders.items()):
            if order['price'] != price:
                continue
            if order['side'] == side:
                continue
            #reduce the queue ahead first
            if order['queue_ahead'] > 0: #that mean there are more order before me that needs to be executed frst FIFO method, we had queue ahead 5
                consume = min(order['queue_ahead'], traded_qty) #it is the remaining orders available: Traded_qty is the available qty the market have, Now market has 6 contratcs
                order['queue_ahead'] -= consume #traded_qty -= 5 → becomes 1
                traded_qty -= consume #order['queue_ahead'] -= 5 → becomes 0
                
            if traded_qty <= 0:
                continue
                
            #now fill your order
            fill_qty = min(order['qty'], traded_qty) #fill_qty is the order I want to execute and it can be happen if the market has the remaining qty, MY qty 2, remaining traded qty 1
            order['qty'] -= fill_qty # 2-1 =1
            traded_qty -= fill_qty #0

            if fill_qty > 0:
                filled_orders.append((oid,fill_qty,price))
            if order['qty'] <= 0:
                del self.my_orders[oid]
        return filled_orders
            

In [5]:
class AggressorFlow: #used to see whether buyers are more or sellers: Tells you who is actually hitting the book.
    def __init__(self,window = 200):
        self.window = window
        self.flow = deque(maxlen = window) #This stores recent aggressive trades. 200 trades what is the recent market pressure

    def update(self,trade_side,qty):
        signed_qty = qty if trade_side == 'BUY' else -qty 
        self.flow.append(signed_qty) #[+10, -7, +5, +20]

    def net_flow(self):
        return sum(self.flow) #10 - 7 + 5 + 20 = +28

    def normalized_flow(self):
        if len(self.flow) == 0: 
            return 0
        total = sum(abs(x) for x in self.flow) 
        if total == 0:
            return 0
        return sum(self.flow) / total  #10 - 7 + 5 + 20 = +42 so 28/42 ans: between -1 to 1 (bullish sentiment) negetive opposite

In [6]:
class MicrostructureAlpha:
    def __init__(self,
                imb_threshold = 0.05,
                delta_threshold = 0.02,
                max_spread = 1.0):
        self.imb_threshold = imb_threshold
        self.delta_threshold = delta_threshold
        self.max_spread = max_spread
        self.prev_imbalance = 0
        self.prev_spread = None
        self.flow = AggressorFlow(window = 150)
        self.flow_threshold = 0.2
        #Calculation on more accurate alpha
        self.mid_history = deque(maxlen = 500)
        self.momentum_window_ms = 500
        self.last_alpha = 0

    def update_trade_flow(self,trade_side,qty):
        self.flow.update(trade_side,qty)
        
    def update_mid(self,mid,ts):
        now = time.time()*1000
        self.mid_history.append((now,mid)) #every mid data with 1000000ms will show
        
    def generate_signal(self,book):
        bid = book.best_bid()
        ask = book.best_ask()

        #if need both sides:
        if bid is None or ask is None:
            return None

        spread = ask - bid
        if spread > self.max_spread:
            return None

        imbalance = book.imbalance()
        delta_imb = imbalance - self.prev_imbalance

        signal = None

        #Buy Condition:
        if(imbalance > self.imb_threshold and
          imbalance > self.delta_threshold and
          (self.prev_spread is None or spread <= self.prev_spread)):
            signal = 'BUY'

        #Sell Condition:
        if(imbalance < -self.imb_threshold and
          imbalance < -self.max_spread and
          (self.prev_spread is None or spread <= self.prev_spread)):
            signal = 'SELL'

        #Update status:
        self.prev_imbalance = imbalance
        self.prev_spread = spread

        return signal

        #short term prediction of price pressure:
    def compute_alpha (self,book):
        now = time.time()*1000

        bid = book.best_bid() #best_bid = 100.00
        ask = book.best_ask() #best_Ask = 100.10

        if bid is None or ask is None:
            return 0
            #Imbalance:
        imbalance = book.imbalance() #suppose book imalance is 0.4 means stronger buy side liquidity (bullish bias)
        flow = self.flow.normalized_flow() #normalised flow gives 0.6 aggressive buyers dominating
        #momentum = 500 ms:
        mom = 0
        recent_mids = [mid for ts, mid in self.mid_history
                      if now - ts <= self.momentum_window_ms] #every mid data between 0 to 500 ms will show
        if len(recent_mids) >= 2:
            mom = recent_mids[-1] - recent_mids[0] #recent_mids = [100.02, 100.04, 100.06] mom = 100.06 - 100.02 = 0.04


        #combine:
        alpha = 0.5*imbalance + 0.3*flow + 0.2*mom
        self.last_alpha = alpha
        return alpha
        
            

In [7]:
#After I get filled, does price usually move against me?
#If YES → market is toxic → reduce trading.

class AdverseSelectionMonitor:
    def __init__(self,window = 50, threshold = 0.002):
        self.window = window
        self.threshold = threshold
        self.pending_fills = []
        self.evaluated_returns = deque(maxlen = window) #stores returns after fill [-0.001, -0.003, +0.0005, -0.002]

    

    def record_fill(self,side,price,timestamp): #'side': 'BUY','price': 100,'timestamp': 1000
        self.pending_fills.append({
            'side': side,
            'price': price,
            'timestamp': timestamp
        })

    def update_mid_price(self,mid,ts): #“After I get filled, does price move in my favor or against me?”
        still_pending = []

        for fill in self.pending_fills:
            if ts - fill['timestamp'] >= 200: #This is a 200ms markout engine. Wait 200ms, Compare new mid price vs execution price, Calculate return
                if fill['side'] == 'BUY':
                    ret = (mid - fill['price']) / fill['price'] #ret = (102 - 100) / 100
                else:
                    ret = (fill['price'] - mid) / fill['price'] 

                self.evaluated_returns.append(ret)

            else:
                still_pending.append(fill)
        self.pending_fills = still_pending

    def is_toxic(self):
        if len(self.evaluated_returns) < self.window:
            return False
            
        avg_ret = sum(self.evaluated_returns) / len(self.evaluated_returns)
        
        return avg_ret < -self.threshold  #threshold = 0.002, avg_ret = -0.003 True market is toxic
        
        

In [8]:
class InventoryManager:
    def __init__(self,max_inventory = 5, max_skew = 0.5):
        self.max_inventory = max_inventory
        self.max_skew = max_skew

    def adjust_signal(self,signal,position):
        if signal is None:
            return None
        if abs(position) >= self.max_inventory:
            return 'SELL' if position > 0 else 'BUY'

        if position > 0 and signal == 'BUY':
            return None
        if position < 0 and signal == 'SELL':
            return None
        return signal

In [9]:
class RiskManager:
    def __init__(self,max_position = 500):
        self.max_position = max_position

    def allow(self,qty,position):
        return abs(position + qty) <= self.max_position

In [10]:
#OMS-Order Management PnL:
class OMS:
    def __init__(self):
        self.position = 0
        self.cash = 0
        self.order_id = 0
        
    def next_id(self):
        self.order_id += 1
        return self.order_id
        
    def on_fill(self,side,price,qty):
        if side == 'BUY':
           self.position += qty
           self.cash -= price * qty
        else:
           self.position -= qty
           self.cash += price * qty
        
    def pnl(self,mid):
        if mid is None:
            return self.cash #Mark to Market
        return self.cash + self.position * mid
        
        

In [11]:
class HFT_Engine:
    def __init__(self):
        self.book = OrderBook()
        self.strategy = MicrostructureAlpha()
        self.risk = RiskManager()
        self.oms = OMS()
        self.adverse = AdverseSelectionMonitor()
        self.cooldown = 0
        self.inventory = InventoryManager(max_inventory=5)
        #two sided market:
        self.active_orders = {
            "BUY": None,
            "SELL": None
        }

        self.base_spread = 0.5
        self.base_size = 1
        self.tick_size = 0.01
        #Latency settings:
        self.market_latency_ms = 20
        self.decision_latency_ms = 5
        self.order_latency_ms = 6
        #backtest:
        # Rolling metrics window (last N observations)
        self.metrics_window = 5000   # adjust if needed
        self.last_snapshot_time = time.time()
        self.snapshot_interval = 10  # seconds
        
        self.pnl_history = []
        self.position_history = []
        self.mid_history = []
        self.timestamp_history = []
        self.trade_log = []
        self.order_count = 0
        self.fill_count = 0
        #volatility Estimator:
        self.mid_returns = deque(maxlen = 500)
        self.volatility = 0
        self.vol_multiplier = 20
        #Market Make concept:
        self.inventory_skew_coeff = 0.02
        self.inv_spread_coeff = 0.05
        self.max_position = 5
        self.toxic_spread_multiplier = 2
        #realised spread and markout Tracking:
        self.markout_horizon = 30
        self.realized_spread = []
        self.pending_markouts = []
        #Inventory aversion:
        self.inventory_risk_aversion = 0.05
        #priority queue for pending order:
        self.pending_order = []

    def schedule_order(self,ts,side,price,qty):
        execute_ts = ts + self.decision_latency_ms + self.order_latency_ms
        heapq.heappush(self.pending_order, (execute_ts,side,price,qty))
        self.order_count += 1

    def process_delayed_orders(self,current_ts):
        while self.pending_order and self.pending_order[0][0] <= current_ts:
            _,side,price,qty = heapq.heappop(
                self.pending_order
            )
            # place into queue (NOT instant fill)
            oid = self.oms.next_id()

            self.book.place_limit_order(
                oid,
                side,
                price,
                qty
            )

            print(
                f"ORDER ARRIVED | {side} @ {price}"
            )
        

    def mid_price(self):
        bid = self.book.best_bid()
        ask = self.book.best_ask()

        if bid is not None and ask is not None:
            return (bid + ask) / 2

        return None

    def process_event(self,event):
        
        ts,side,price,qty,etype = event
        self.current_ts = ts
        self.process_delayed_orders(ts)
        self.book.process(event) #update book
        #interpretate trade events as aggressor flow
        if etype == 'TRADE':
            self.strategy.update_trade_flow(side,qty)
        
        fills = self.book.process_trade(side,price,qty) #simulate trade hitting book
        for oid,fill_qty,fill_price in fills:
            fill_side = 'BUY' if side == 'SELL' else 'SELL' #here I am buying when someone is selling, side represent the other party
            self.oms.on_fill(fill_side,fill_price,fill_qty)
            self.active_orders[fill_side] = None
            self.adverse.record_fill(fill_side, fill_price,ts)
            #for backtest:
            self.fill_count += 1
            self.trade_log.append({
                'timestamp': ts,
                'side': fill_side,
                'price': fill_price,
                'qty': fill_qty,
                'position_after': self.oms.position
            })

            self.pending_markouts.append({
                'ref_index': len(self.mid_history),
                'price': fill_price,
                'side': side
            })

            print(
                f"FILL | qty={fill_qty} @ {fill_price}")
            print(
                f"FILL | qty={fill_qty} @ {fill_price} "
                f"| PnL={self.oms.pnl(self.mid_price()):.2f}"
            )
            
        mid = self.mid_price()
        if mid is not None:
            self.adverse.update_mid_price(mid,ts)
            self.strategy.update_mid(mid,ts)

        if mid is not None and len(self.mid_history) > 0:
            prev = self.mid_history[-1]
            ret = mid - prev
            self.mid_returns.append(ret)

            if len(self.mid_returns)>20:
                self.volatility = np.std(self.mid_returns)
                
        if self.adverse.is_toxic():
            print("⚠ Toxic flow detected — pausing trading")
            return
        if not self.adverse.is_toxic():
            self.market_make(ts)
        
        mid = self.mid_price()
        
        #backtest 
        if mid is not None:
            self.pnl_history.append(self.oms.pnl(mid))
            self.position_history.append(self.oms.position)
            self.mid_history.append(mid)
            self.timestamp_history.append(ts)
            if time.time() - self.last_snapshot_time > self.snapshot_interval:
                self.compute_metrics()
                self.last_snapshot_time = time.time()

        #markout computation:
        new_pending = []
        for trade in self.pending_markouts:
            ref = trade['ref_index']
            if len(self.mid_history) > ref + self.markout_horizon:
                future_mid = self.mid_history[ref + self.markout_horizon]
                direction = 1 if trade['side'] == 'BUY' else -1
                markout = direction * (future_mid - trade['price'])
                self.realized_spread.append(markout)

            else:
                new_pending.append(trade)

        self.pending_markouts = new_pending

                
    def market_make(self,ts): #this functions determines that due to more latency it may not take te price it desired, so we take a fair value of it and then will accept or reject the price
        best_bid = self.book.best_bid()
        best_ask = self.book.best_ask()

        if best_bid is None or best_ask is None:
            return None
            
        
        mid = (best_bid + best_ask) / 2
        alpha = self.strategy.compute_alpha(self.book)#In microstructure alpha, mom defined
        inventory_skew = - self.oms.position * self.inventory_skew_coeff #core market maker logic
        fair_value = mid + alpha + inventory_skew

        #spread logic: Avellaneda-Stoikov Model
        vol_spread = self.vol_multiplier * self.volatility #volatility rises, spreads widen
        spread = max(self.base_spread, vol_spread)

        #Widen spread when inventory large:
        spread += self.inv_spread_coeff * abs(self.oms.position)

        if self.adverse.is_toxic():
            self *= self.toxic_spread_multiplier

        #Soft Inventory Limit:
        inventory_ratio = self.oms.position / self.max_position
        fair_value -= inventory_ratio * spread
        
        #Final quotes:
        bid_price = round(fair_value - spread / 2,2)
        ask_price = round(fair_value + spread / 2,2)

        size = self.base_size

        #risk check:        
        current_pos = self.oms.position
        
        if self.risk.allow(size,current_pos):
            self._update_side('BUY',bid_price,size)

        if self.risk.allow(-size, current_pos):
            self._update_side('SELL',ask_price,size)
        
        print(
        f"MID={mid:.2f} | POS={self.oms.position} | "
        f"BID={bid_price:.2f} | ASK={ask_price:.2f}")
        print(f"ALPHA={alpha:.5f}")
    
    def _update_side(self, side, new_price, size):

     active = self.active_orders[side]

     if active is None:
         oid = self.oms.next_id()
         self.active_orders[side] = {
            'id': oid,
            'price': new_price
        }
         
         self.schedule_order(
             ts=self.current_ts,
             side=side,
             price=new_price,
             qty=size
        )
         return

     if abs(active['price'] - new_price) >= self.tick_size:
         self.book.cancel_order(active['id'])

         oid = self.oms.next_id()
         self.active_orders[side] = {
            'id': oid,
            'price': new_price 
         }

         self.schedule_order(
             ts=self.current_ts,
             side=side,
             price=new_price,
             qty=size
         )
    #Backtesting
    def compute_metrics(self):
        pnl = np.array(self.pnl_history[-self.metrics_window:])

        if len(pnl) < 2:
            print('Not enough data')
            return

        returns = np.diff(pnl)
        mean_ret = np.mean(returns)
        vol = np.std(returns)

        sharpe = 0
        if vol > 0:
            sharpe = mean_ret / vol if vol > 0 else 0

         #Max dd:
        peak = np.maximum.accumulate(pnl)
        drawdown = pnl - peak
        max_dd = np.min(drawdown)
        fill_ratio = 0
        if self.order_count > 0:
            fill_ratio = self.fill_count / self.order_count

        if len(self.realized_spread) > 0:
            avg_rs = np.mean(self.realized_spread)
            print(f"Average Markout ({self.markout_horizon} events): {avg_rs:.5f}")

        print("\n========== PERFORMANCE ==========")
        print(f"Final PnL: {pnl[-1]:.2f}")
        print(f"Sharpe: {sharpe:.3f}")
        print(f"Max Drawdown: {max_dd:.2f}")
        print(f"Total Orders: {self.order_count}")
        print(f"Total Fills: {self.fill_count}")
        print(f"Fill Ratio: {fill_ratio:.3f}")
        print("=================================\n")

    def snapshot_metrics(self):
        print("\n--- LIVE SNAPSHOT ---")
        self.compute_metrics()


In [12]:
!pip install websocket-client pandas
import json
import threading
import websocket
import time


[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
class BinanceFeed:
    def __init__(self, engine, symbol="btcusdt"):
        self.engine = engine
        self.symbol = symbol.lower()

        self.url = (
            f"wss://stream.binance.com:9443/"
            f"stream?streams={self.symbol}@depth@100ms/"
            f"{self.symbol}@trade"
        )

    def on_message(self, ws, message):
        msg = json.loads(message)

        stream = msg.get("stream")
        data   = msg.get("data")

        if not stream or not data:
            return

        ts = int(data.get("E", time.time()*1000))

        # ----------------------
        # DEPTH EVENTS
        # ----------------------
        if "depth" in stream:

            # Bids
            for price, qty in data.get("b", []):
                price = float(price)
                qty   = float(qty)

                etype = "NEW" if qty > 0 else "CANCEL"

                event = (ts, "BUY", price, qty, etype)
                self.engine.process_event(event)

            # Asks
            for price, qty in data.get("a", []):
                price = float(price)
                qty   = float(qty)

                etype = "NEW" if qty > 0 else "CANCEL"

                event = (ts, "SELL", price, qty, etype)
                self.engine.process_event(event)

        # ----------------------
        # TRADE EVENTS
        # ----------------------
        elif "trade" in stream:

            price = float(data["p"])
            qty   = float(data["q"])

            # m = True → seller aggressor
            side = "BUY" if data["m"] else "SELL"

            event = (ts, side, price, qty, "TRADE")

            self.engine.process_event(event)
    print("message received")
    def on_error(self, ws, error):
        print("WebSocket error:", error)

    def on_close(self, ws, close_status_code, close_msg):
        print("WebSocket closed")

    def on_open(self, ws):
        print("Connected to Binance")

    def start(self):
        ws = websocket.WebSocketApp(
            self.url,
            on_message=self.on_message,
            on_error=self.on_error,
            on_close=self.on_close,
            on_open=self.on_open
        )

        thread = threading.Thread(target=ws.run_forever)
        thread.daemon = True
        thread.start()

message received


In [ ]:
engine = HFT_Engine()
feed = BinanceFeed(engine)

feed.start()

print("🚀 Starting HFT Engine...")
while True:
    time.sleep(1)


🚀 Starting HFT Engine...
Connected to Binance
MID=67750.07 | POS=0 | BID=67749.32 | ASK=67749.82
ALPHA=-0.49518
MID=67750.07 | POS=0 | BID=67749.32 | ASK=67749.82
ALPHA=-0.49533
MID=67750.07 | POS=0 | BID=67749.32 | ASK=67749.82
ALPHA=-0.49533
MID=67750.07 | POS=0 | BID=67749.32 | ASK=67749.82
ALPHA=-0.49538
MID=67750.07 | POS=0 | BID=67749.32 | ASK=67749.82
ALPHA=-0.49538
MID=67750.07 | POS=0 | BID=67749.32 | ASK=67749.82
ALPHA=-0.49538
MID=67750.07 | POS=0 | BID=67749.32 | ASK=67749.82
ALPHA=-0.49756
MID=67750.07 | POS=0 | BID=67749.32 | ASK=67749.82
ALPHA=-0.49756
MID=67750.07 | POS=0 | BID=67749.32 | ASK=67749.82
ALPHA=-0.49756
MID=67750.07 | POS=0 | BID=67749.32 | ASK=67749.82
ALPHA=-0.49756
MID=67750.07 | POS=0 | BID=67749.32 | ASK=67749.82
ALPHA=-0.49756
MID=67750.07 | POS=0 | BID=67749.32 | ASK=67749.82
ALPHA=-0.49759
MID=67750.07 | POS=0 | BID=67749.32 | ASK=67749.82
ALPHA=-0.49759
ORDER ARRIVED | BUY @ 67749.32
ORDER ARRIVED | SELL @ 67749.82
MID=67750.07 | POS=0 | BID=67749.

In [ ]:
#✅ Correct Flow Architecture
#Step 1 — Market sends event
#(ts, side, price, qty, etype)


#Example:

#(1234567, 'BUY', 65000, 0.5, 'TRADE')

#Step 2 — Engine processes it
#self.strategy.update_trade_flow(side, qty)
#signal = self.strategy.generate_signal(...)

#Step 3 — If signal exists
#if signal == "BUY":
 #   place order